In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
#dataset = '../Dataset/Car/car'
#dataset = '../Dataset/Balance-sheet/train.txt'
#dataset = '../Dataset/Hayes-Roth/Hayes-Roth'
dataset = '../Dataset/Heart-Data/train.txt'
#dataset = '../Dataset/TicTacToe/tic tac toe'
#dataset = '../Dataset/adult.data'
#dataset = '../Dataset/mush.data'
#dataset = '../Dataset/nursery.data'
DATASET_SPLIT_RATIO = .1
np.random.seed(2)

In [3]:
def clasification_report(y_true, y_pred, output_dict):
    unique_labels = list(np.unique(y_true))
    ret  = {}
    for label in unique_labels:
        TP = np.sum((np.array(y_true) == label) & (np.array(y_true) == np.array(y_pred)))
        TN = np.sum((np.array(y_true) != label) & (np.array(y_true) == np.array(y_pred)))
        FP = np.sum((np.array(y_true) == label) & (np.array(y_true) != np.array(y_pred)))
        FN = np.sum((np.array(y_true) != label) & (np.array(y_true) != np.array(y_pred)))
    ret['precision'] = TP / (TP + FP)
    ret['recall'] = TP / (TP+FN)
    ret['f1-measure'] = 2 * ((ret['precision'] * ret['recall']) / (ret['precision'] + ret['recall']))
    return ret

In [4]:
class NaiveBayesClassifier:
    def __init__(self):
        print('Initiated')
    def train(self, data):
        self.train_data = data
        self.list_of_features = list(data)
        self.list_of_labels = np.unique(data[self.list_of_features[-1]], return_counts=True)[0]
        self.list_of_labels_count = np.unique(data[self.list_of_features[-1]], return_counts=True)[1]
    def predict(self, data):
        mx = -1000
        mx_idx = -1
        for label_itr in range(len(self.list_of_labels)):
            probability = np.log(self.list_of_labels_count[label_itr]/len(self.train_data))
            for i in range(len(self.list_of_features) - 1):
                #print(self.train[i].head())
                #print(data[i])
                #print(type(self.train[i]))
                #print(type(data[i]))
                #print(data)
                #print(self.train[i].index)
                #print(data[i].index)
                #print(self.train[i].sort_index().sort_index(axis=1) == data[i].sort_index().sort_index(axis=1))
                #print(self.train[i] == data[i])
                #match = self.train[(self.train[i] == data[i]) & (self.train[self.list_of_features[-1]] == self.list_of_labels[label_itr])]
                match = dict(self.train_data.groupby([i, self.list_of_features[-1]]).size())
                #print(match)
                try:
                    matches = match[(data[i].values[0], self.list_of_labels[label_itr])]
                except:
                    matches = 0
                probability += np.log(((matches+1)/(self.list_of_labels_count[label_itr] + len(np.unique(data[i])))))
            if mx_idx == -1 or probability > mx:
                mx = probability
                mx_idx = label_itr
        return self.list_of_labels[mx_idx]
    def test(self, test):
        right_predictions = 0
        y_true = []
        y_pred = []
        for i in range(len(test)):
            y_pred.append(self.predict(test[i:i+1]))
            y_true.append(test[i:i+1][list(test)[-1]].values[0])
            if self.predict(test[i:i+1]) == test[i:i+1][list(test)[-1]].values[0]:
                right_predictions+=1
        ret_dict = classification_report(y_true, y_pred, output_dict=True)
        ret_dict['accuracy'] = round(right_predictions/len(test) * 100, 2)
        return ret_dict

In [5]:
def cross_validation(dataframe):
    important_classifier = NaiveBayesClassifier()
    accuracies = []
    data_in_a_single_fold = int(np.ceil(max(10, len(dataframe)/10)))
    #data_in_a_single_fold = int(np.ceil(.1*len(dataframe)))
    number_of_folds = int(np.ceil(len(dataframe)/data_in_a_single_fold))
    print('Number of folds:', number_of_folds)
    print('Number of rows in a single fold:', data_in_a_single_fold)
    for i in range(number_of_folds):
        #print(min((i)*data_in_a_single_fold, len(dataframe)), 'bal')
        a = dataframe.iloc[0 : min((i)*data_in_a_single_fold, len(dataframe))]

        #print('min', min((i+1) * data_in_a_single_fold, len(dataframe)))
        b = dataframe.iloc[min((i+1) * data_in_a_single_fold, len(dataframe)) :  len(dataframe)]
        #print(b.head())
        train_data = pd.concat([a, b]).reset_index(drop=True)
        
        test = dataframe.iloc[i * data_in_a_single_fold : min((i+1)*data_in_a_single_fold, len(dataframe))].reset_index(drop=True)
        print('Number of training samples:', len(train_data))
        print('Number of test samples:', len(test))
        print('Labels:', np.unique(train_data[list(dataframe)[-1]]))
        #print(train_data.head())
        important_classifier.train(train_data)    
        accuracies.append(important_classifier.test(test))
    return accuracies

In [6]:
dataframe = pd.read_csv(dataset, header=None)
dataframe = dataframe.sample(frac=1)
accuracies = cross_validation(dataframe)
#print('Accuracies:', accuracies)
#print('Average accuracy:', round(np.mean(accuracies), 2))

Initiated
Number of folds: 10
Number of rows in a single fold: 19
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]


/home/immuntasir/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 171
Number of test samples: 16
Labels: [0 1]


In [7]:
avg_wei = {'recall':0, 'f1-score':0, 'precision':0}
avg_acc = 0
n = 0
for accu in accuracies:
    for key in avg_wei.keys():
        avg_wei[key] += accu['weighted avg'][key]
    avg_acc += accu['accuracy']
    n += 1
    
for key in avg_wei.keys():
    avg_wei[key] /= n
    avg_wei[key] = round(avg_wei[key], 2)
avg_acc /= n
avg_acc = round(avg_acc, 2)

In [8]:
print('Average accuracy:', avg_acc)

Average accuracy: 77.5


In [9]:
y_true = [1, 3, 2, 1, 2, 3, 1]
y_pred = [1, 1, 1, 2, 2, 2, 3]

Car Dataset: 93.06%
Balance Sheet: 26.98%
Hayes-Roth: 50.0%
Heart Data: 89.47%
TicTacToe: 85.42%

In [10]:
(np.array(y_true) == np.array(y_pred))

array([ True, False, False, False,  True, False, False])

In [11]:
np.sum((np.array(y_true) == 1) & (np.array(y_true) == np.array(y_pred)))

1

In [14]:
accuracies

[{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
  '1': {'precision': 1.0,
   'recall': 0.8421052631578947,
   'f1-score': 0.9142857142857143,
   'support': 19},
  'micro avg': {'precision': 0.8421052631578947,
   'recall': 0.8421052631578947,
   'f1-score': 0.8421052631578947,
   'support': 19},
  'macro avg': {'precision': 0.5,
   'recall': 0.42105263157894735,
   'f1-score': 0.45714285714285713,
   'support': 19},
  'weighted avg': {'precision': 1.0,
   'recall': 0.8421052631578947,
   'f1-score': 0.9142857142857143,
   'support': 19},
  'accuracy': 84.21},
 {'0': {'precision': 0.5,
   'recall': 0.6666666666666666,
   'f1-score': 0.5714285714285715,
   'support': 3},
  '1': {'precision': 0.9333333333333333,
   'recall': 0.875,
   'f1-score': 0.9032258064516129,
   'support': 16},
  'micro avg': {'precision': 0.8421052631578947,
   'recall': 0.8421052631578947,
   'f1-score': 0.8421052631578947,
   'support': 19},
  'macro avg': {'precision': 0.716666666666666

In [13]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
48,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
79,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,1,1
53,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
12,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
115,0,0,1,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
